In [ ]:
df=read.csv("C:/Users/student/Downloads/R/adult.csv", header=FALSE)
names(df)=c('age','workclass','fnlwt','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','target')
str(df)


## Assignment 11

1. Replace all the missing values by NA

sum(is.na(df))
df[df==" ?"]=NA
sum(is.na(df))


2.	Replace the categorical missing values by the mode of the corresponding variables

misshandled=function(data){
  for(i in 1:ncol(data)){
    if (!is.numeric(data[,i])){
      levels=unique(data[,i])
      data[,i][is.na(data[,i])]=levels[which.max(tabulate(match(data[,i],levels)))]
    }
  }
  return(data)
}
newdf=misshandled(df)
sum(is.na(newdf))


3.	Replace the numeric missing values by the median using caret


library(caret)
preProcess_missingdata_model <- preProcess(newdf, method='medianImpute')
preProcess_missingdata_model


4.	The variable "native country" has 16 categories. Group the categories of this variable into 5 categories.  Do the same for all categorical variables with more than 10 categories. 


levels(newdf$education)=c("Before HS","Before HS", "Before HS", "Before HS","Before HS","Before HS","Before HS","Before HS","Some College", "Some College", "Bachelors", "Higher College", "Some HS", "Higher College","Before HS","Higher College","Some College")
                  

levels(newdf$native_country)=c("Asia","Asia", "North America", "Asia", "South America", "Central America", "Central America", "South America", "Central America", "Europe", "Europe", "Europe", "Europe", "Central America", "Central America", "Europe", "Central America", "Asia", "Europe", "Asia", "Asia", "Europe", "Europe", "Central America", "Asia", "Asia", "North America", "Central America", "North America", "South America", "Asia", "Europe", "Europe","Central America", "Europe", "Asia", "Asia", "Asia", "Central America", "North America", "Asia", "Europe")

levels(newdf$occupation)=c("Craft","Office", "Army","Craft", "Office","Craft", "Craft", "Craft", "Other", "Craft", "Craft", "Army", "Sales", "Craft", "Craft")



5.	Encoding categorical variable using one hot encoding (dummy encoding) 


trainData <- predict(preProcess_missingdata_model, newdata =newdf)
dummies_model <- dummyVars(target ~ ., data=trainData)
trainData_mat <- predict(dummies_model, newdata = trainData)

6.	Scale and center the data 


preProcess_missingdata_model <- preProcess(newdf, method= c("center", "scale"))
preProcess_missingdata_model


7.	Split the data into training (70%) and testing (30%) with the seeding set to be 2018 [including the code: set.seed(2018)]. Build a decision tree and report the accuracy and balanced accuracy


set.seed(2018)
splitIndex<- createDataPartition(newdf$target, p=.7, list=FALSE, times = 1)
train<- newdf[splitIndex,]
test<- newdf[-splitIndex,]

8.	Build a random forest using the ranger package.  Report the accuracy and balanced accuracy

library(ranger)
model = ranger(target ~., data = train)
pred  = predict(model, data = test)$predictions
levels(test$target) = c("0", "1")
levels(pred) = c("0", "1")
cm=confusionMatrix(pred, test$target, positive="1")
cm

#9.	Redo 3 with missing values being replaced by the mean.  Rebuild the model and report the models' performances (the accuracy and balanced accuracy). 
df1=df
levels(df1$education)=c("Before HS","Before HS", "Before HS", "Before HS","Before HS","Before HS","Before HS","Before HS","Some College", "Some College", "Bachelors", "Higher College", "Some HS", "Higher College","Before HS","Higher College","Some College")
                  
levels(df1$native_country)=c("Asia","Asia", "North America", "Asia", "South America", "Central America", "Central America", "South America", "Central America", "Europe", "Europe", "Europe", "Europe", "Central America", "Central America", "Europe", "Central America", "Asia", "Europe", "Asia", "Asia", "Europe", "Europe", "Central America", "Asia", "Asia", "North America", "Central America", "North America", "South America", "Asia", "Europe", "Europe","Central America", "Europe", "Asia", "Asia", "Asia", "Central America", "North America", "Asia", "Europe")

levels(df1$occupation)=c("Craft","Office", "Army","Craft", "Office","Craft", "Craft", "Craft", "Other", "Craft", "Craft", "Army", "Sales", "Craft", "Craft")


missingmean=function(data){
  for(i in 1:ncol(data)){
    if (is.numeric(data[,i])){
      data[,i][is.na(data[,i])]=mean(data[,i], na.rm=TRUE)
    } else{
      levels=unique(data[,i])
      data[,i][is.na(data[,i])]=levels[which.max(tabulate(match(data[,i],levels)))]
    }
  }
  return(data)
}
df2 = missingmean(df1)
sum(is.na(df2))


trainData <- predict(preProcess_missingdata_model, newdata =df1)
dummies_model <- dummyVars(target ~ ., data=trainData)
trainData_mat <- predict(dummies_model, newdata = trainData)

set.seed(2018)
splitIndex<- createDataPartition(df1$target, p=.7, list=FALSE, times = 1)
train<- df2[splitIndex,]
test<- df2[-splitIndex,]

model1 = ranger(target ~., data = train)
pred1  = predict(model1, data = test)$predictions
levels(test$target) = c("0", "1")
levels(pred1) = c("0", "1")
cm1=confusionMatrix(pred1, test$target, positive="1")
cm1

#10.	Redo 3 with missing values being replaced by the "knn" imputation method.  Rebuild the model and report the models' performances (the accuracy and balanced accuracy)


preProcess_missingdata_model <- preProcess(newdf, method='knnImpute')
set.seed(2018)
splitIndex<- createDataPartition(newdf$target, p=.7, list=FALSE, times = 1)
train<- newdf[splitIndex,]
test<- newdf[-splitIndex,]
model2 = ranger(target ~., data = train)
pred2  = predict(model2, data = test)$predictions
levels(test$target) = c("0", "1")
levels(pred2) = c("0", "1")
cm2=confusionMatrix(pred2, test$target, positive="1")
cm2

#11.	Redo 6 with only scaling and centering the non-encoded variables. Rebuild the model and report the models' performances (the accuracy and balanced accuracy)

preProcess_missingdata_model <- preProcess(newdf, method=c("center", "scale"))
trainData <- predict(preProcess_missingdata_model, newdata =newdf)
dummies_model <- dummyVars(target ~ ., data=trainData)
trainData_mat <- predict(dummies_model, newdata = trainData)
set.seed(2018)
splitIndex<- createDataPartition(newdf$target, p=.7, list=FALSE, times = 1)
train<- newdf[splitIndex,]
test<- newdf[-splitIndex,]
model3 = ranger(target ~., data = train)
pred3  = predict(model3, data = test)$predictions
levels(test$target) = c("0", "1")
levels(pred3) = c("0", "1")
cm3=confusionMatrix(pred3, test$target, positive="1")
cm3
```


ERROR: Error in parse(text = x, srcfile = src): <text>:1:9: unexpected symbol
1: 10.     Redo
            ^
